### Hamming Distance, Edit Distance, Dynamic Programming and Allignments

In [6]:
import numpy as np

In [7]:
def hammingDistance(x, y):
    assert len(x) == len(y)
    hm = 0
    for i in range(len(x)):
        if x[i] != y[i]:
            hm += 1
    return hm

In [8]:
x = 'aaa'
y = 'bbb'
print hammingDistance(x,y)


3


In [9]:
def editDistance_recursive(x, y):
    if len(x) == 0:
        return len(y)
    if len(y) == 0:
        return len(x)
    delta = 1
    if x[-1] == y[-1]:
        delta = 0
    diagonal = editDistance_recursive(x[:-1], y[:-1]) + delta
    vertical = editDistance_recursive(x[:-1], y) + delta
    horizontal = editDistance_recursive(x, y[:-1]) + delta
    return min(diagonal, vertical, horizontal)

In [10]:
%%time 
editDistance_recursive('Shakespeare', 'shake spear') 

CPU times: user 26.4 s, sys: 281 ms, total: 26.6 s
Wall time: 27.8 s


3

In [19]:
def editDistance_dynamic(x, y):
    D = np.zeros((len(x)+1, len(y)+1), dtype=int)
    D[0, 0:] = range(0, len(y)+1)
    D[0:, 0] = range(0, len(x)+1)
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            delt = 1 if x[i-1] != y[j-1] else 0
            D[i, j] = min(D[i-1, j-1]+delt, D[i-1, j]+1, D[i, j-1]+1)
    return D[len(x), len(y)], D

In [36]:
%%time
x = 'GCGTATGCACGC'
y = 'GCTATGCCACGC'
distance, D = editDistance_dynamic(x, y)
print D
print distance

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 1  0  1  2  3  4  5  6  7  8  9 10 11]
 [ 2  1  0  1  2  3  4  5  6  7  8  9 10]
 [ 3  2  1  1  2  3  3  4  5  6  7  8  9]
 [ 4  3  2  1  2  2  3  4  5  6  7  8  9]
 [ 5  4  3  2  1  2  3  4  5  5  6  7  8]
 [ 6  5  4  3  2  1  2  3  4  5  6  7  8]
 [ 7  6  5  4  3  2  1  2  3  4  5  6  7]
 [ 8  7  6  5  4  3  2  1  2  3  4  5  6]
 [ 9  8  7  6  5  4  3  2  2  2  3  4  5]
 [10  9  8  7  6  5  4  3  2  3  2  3  4]
 [11 10  9  8  7  6  5  4  3  3  3  2  3]
 [12 11 10  9  8  7  6  5  4  4  3  3  2]]
2
CPU times: user 2.29 ms, sys: 1.22 ms, total: 3.51 ms
Wall time: 2.34 ms


In [60]:
def trace_editDistance(x, y, D):
    edit_transcript = []
    allignment_x = []
    i = D.shape[0]-1
    j = D.shape[1]-1
    while i > 0 and j > 0:
        diagonal, vertical, horizontal = D[i-1,j-1], D[i-1, j], D[i, j-1]
        minimum = min(diagonal, vertical, horizontal)
        if minimum == diagonal:
            allignment_x.append(x[i-1])
            i = i-1
            j = j-1
        elif minimum == horizontal:
            allignment_x.append('-')
            j = j-1
        else:
            allignment_x.append(x[i-1])
            i = i-1
    return (''.join(allignment_x))[::-1]

In [61]:
trace_editDistance(x, y, D)

'GCGTATGC-ACGC'

In [13]:
def calculate_penalty(x, y):
    # if they are the same, return 0
    if x == y:
        return 0
    # if one of them is empty, return 8
    if x == '-' or y == '-':
        return 8
    min_ = min(x,y)
    max_ = max(x,y)
    if min_ == "A" and max_ == "G":
        return 2
    if min_ == "C" and max_ == "T":
        return 2
    return 4    

In [14]:
def calculate_reward(x, y):
    # if they are the same, return 0
    if x == y:
        return 2
    # if one of them is empty, return 8
    if x == '-' or y == '-':
        return -6
    return -4   

In [15]:
def globalAllignment(x, y):
    D = np.zeros((len(x)+1, len(y)+1), dtype=int)
    D[0,:] = map((lambda x: x*8), range(0, len(y)+1))
    D[:,0] = map((lambda x: x*8), range(0, len(x)+1))
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            delta = calculate_penalty(x[i-1], y[j-1])
            D[i, j] = min(D[i-1, j-1] + calculate_penalty(x[i-1], y[j-1]), 
                          D[i, j-1] + calculate_penalty(x[i-1], '-'), 
                          D[i-1, j] + calculate_penalty('-', y[j-1]))
    return D, D[-1,-1]

In [34]:
x = 'GCGTATGCACGC'
y = 'GCTATGCCACGC'
D, optimal_global_allginment = globalAllignment(x, y)


In [35]:
D

array([[ 0,  8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96],
       [ 8,  0,  8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88],
       [16,  8,  0,  8, 16, 24, 32, 40, 48, 56, 64, 72, 80],
       [24, 16,  8,  4, 10, 18, 24, 32, 40, 48, 56, 64, 72],
       [32, 24, 16,  8,  8, 10, 18, 26, 34, 42, 50, 58, 66],
       [40, 32, 24, 16,  8, 12, 12, 20, 28, 34, 42, 50, 58],
       [48, 40, 32, 24, 16,  8, 16, 14, 22, 30, 36, 44, 52],
       [56, 48, 40, 32, 24, 16,  8, 16, 18, 24, 32, 36, 44],
       [64, 56, 48, 40, 32, 24, 16,  8, 16, 22, 24, 32, 36],
       [72, 64, 56, 48, 40, 32, 24, 16, 12, 16, 24, 26, 34],
       [80, 72, 64, 56, 48, 40, 32, 24, 16, 16, 16, 24, 26],
       [88, 80, 72, 64, 56, 48, 40, 32, 24, 18, 20, 16, 24],
       [96, 88, 80, 72, 64, 56, 48, 40, 32, 26, 18, 24, 16]])

In [ ]:
def trace_globalAllignment(x, y, D):
    i = D.shape[0]
    j = D.shape[1]
    while i > 0 and j > 0:
        
    

In [17]:
def localAllignment(x, y):
    D = np.zeros((len(x)+1, len(y)+1), dtype=int)
    D[0,:] = map((lambda x: x*8), range(0, len(y)+1))
    D[:,0] = map((lambda x: x*8), range(0, len(x)+1))
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            delta = calculate_reward(x[i-1], y[j-1])
            D[i, j] = max(D[i-1, j-1] + calculate_reward(x[i-1], y[j-1]), 
                          D[i, j-1] + calculate_reward(x[i-1], '-'), 
                          D[i-1, j] + calculate_reward('-', y[j-1]))
    return D, D[-1,-1]
    

In [18]:
localAllignment('GGTATGCTGGCGCTA', 'TATATGCGGCGTTT')

(array([[  0,   8,  16,  24,  32,  40,  48,  56,  64,  72,  80,  88,  96,
         104, 112],
        [  8,   2,  10,  18,  26,  34,  42,  50,  58,  66,  74,  82,  90,
          98, 106],
        [ 16,  10,   4,  12,  20,  28,  36,  44,  52,  60,  68,  76,  84,
          92, 100],
        [ 24,  18,  12,   6,  14,  22,  30,  38,  46,  54,  62,  70,  78,
          86,  94],
        [ 32,  26,  20,  14,   8,  16,  24,  32,  40,  48,  56,  64,  72,
          80,  88],
        [ 40,  34,  28,  22,  16,  10,  18,  26,  34,  42,  50,  58,  66,
          74,  82],
        [ 48,  42,  36,  30,  24,  18,  12,  20,  28,  36,  44,  52,  60,
          68,  76],
        [ 56,  50,  44,  38,  32,  26,  20,  14,  22,  30,  38,  46,  54,
          62,  70],
        [ 64,  58,  52,  46,  40,  34,  28,  22,  16,  24,  32,  40,  48,
          56,  64],
        [ 72,  66,  60,  54,  48,  42,  36,  30,  24,  18,  26,  34,  42,
          50,  58],
        [ 80,  74,  68,  62,  56,  50,  44,  38,  32,  26,  